In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


folder = r"C:\Users\Rajath\Desktop\Face-Mask-Detection-master\dataset\train"
category = ["mask", "no_mask"]


x = []
y = []

for ca in category:
    path = os.path.join(folder, ca)
    for im in os.listdir(path):
    	im_path = os.path.join(path, im)
    	img = load_img(im_path, target_size=(224, 224))
    	img = img_to_array(img)
    	img = preprocess_input(img)

    	x.append(img)
    	y.append(ca)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
y = lb.fit_transform(y)
y = to_categorical(y)
x = np.array(x, dtype="float32")
y = np.array(y)

(x_train, x_test, y_train, y_test) = train_test_split(x, y,
	test_size=0.20, stratify=y, random_state=42)

# construct the training image generator for data augmentation
augment = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

E:\anacond3_9\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [6]:
from tensorflow.keras.models import load_model
m2 = load_model("vgg16.model")
m1=load_model("mobilenetv2.model")

In [7]:
pred1 = m1.predict(x_test, batch_size=32)
pred2 = m2.predict(x_test, batch_size=32)

30/30 [==============================] - 372s 12s/step


In [19]:
list1 =sum(pred1[:,0] + pred2[:,0])
list1/2

352.5117207805306

In [12]:
pred1

array([[9.9999857e-01, 1.4739564e-06],
       [6.8865402e-06, 9.9999309e-01],
       [3.3666131e-01, 6.6333872e-01],
       ...,
       [1.0923641e-02, 9.8907638e-01],
       [6.7912021e-07, 9.9999928e-01],
       [9.9999726e-01, 2.7443475e-06]], dtype=float32)

In [22]:
pred2

array([[0.65455776, 0.3454423 ],
       [0.35165533, 0.64834464],
       [0.13911591, 0.86088413],
       ...,
       [0.00514391, 0.9948561 ],
       [0.03330072, 0.96669924],
       [0.99647826, 0.00352173]], dtype=float32)

In [23]:
pred3=pred1+pred2

In [27]:
pred3/2

array([[0.82727814, 0.17272188],
       [0.17583111, 0.82416886],
       [0.2378886 , 0.7621114 ],
       ...,
       [0.00803378, 0.99196625],
       [0.0166507 , 0.98334926],
       [0.9982377 , 0.00176224]], dtype=float32)

In [28]:
pred3 = np.argmax(pred3, axis=1)


print(classification_report(y_test.argmax(axis=1), pred3,target_names=lb.classes_))


              precision    recall  f1-score   support

        mask       0.99      0.98      0.98       376
     no_mask       0.98      0.99      0.99       557

    accuracy                           0.98       933
   macro avg       0.99      0.98      0.98       933
weighted avg       0.99      0.98      0.98       933

